In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    criterion: str
    max_depth: str
    random_state: int
    params : any
    target_column: str

In [ ]:
from src.ML_LAB.constants import * 
from src.ML_LAB.utils.common import read_yaml, create_directories
from src.ML_LAB.entity import * 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_yaml_path,
        params_filepath = params_yaml_path,
        schema_filepath = schema_yaml_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier # Changed to RF
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            criterion=params.criterion,
            max_depth=params.max_depth,
            random_state=params.random_state,
            target_column=schema.name,
            params=self.params.RandomForestClassifier
        )
        return model_trainer_config

In [ ]:
import os
from src.ML_LAB import logger
from sklearn.ensemble import RandomForestClassifier # Changed import
import joblib
from src.ML_LAB.entity import *

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # 1. Load Data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # 2. Split X and y
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)

        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # 3. Initialize Random Forest with params from config
        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            criterion=self.config.criterion,
            max_depth=self.config.max_depth,
            random_state=self.config.random_state
        )

        # 4. Train Model
        rf.fit(train_x, train_y)

        # 5. Save Model
        save_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(rf, save_path)
        
        logger.info(f"Random Forest Model trained and saved at: {save_path}")

In [ ]:
# pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_tr_config = ModelTrainer(config=model_trainer_config)
    model_tr_config.train()
except Exception as e:
    raise e